<a href="https://colab.research.google.com/github/datajcthemax/playdata/blob/main/airflow5_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.utils.trigger_rule import TriggerRule
from datetime import datetime, timedelta

default_args = {
Expand All
	@@ -15,13 +16,14 @@
    schedule_interval='*/5 * * * *',
)

# Task 1: Copy web.log from pd24 to local environment
copy_web_log_task = BashOperator(
    task_id='copy_web_log',
    bash_command="""
    aws s3 cp s3://pd24/web.log /opt/airflow/dags/
    """,
    retries=3,
    retry_delay=timedelta(minutes=1),
    dag=dag
)

Expand All
	@@ -33,10 +35,12 @@
    sort /opt/airflow/dags/RAW.log | uniq -c > /opt/airflow/dags/SUM.log
    touch /opt/airflow/dags/DONE
    """,
    retries=2,
    retry_delay=timedelta(minutes=5),
    trigger_rule=TriggerRule.ONE_SUCCESS,
    dag=dag
)

# Task 3: Send RAW.log, SUM.log, and DONE flag to jai folder in pd24
send_logs_task = BashOperator(
    task_id='send_logs',
Expand All
	@@ -45,10 +49,13 @@
    aws s3 cp /opt/airflow/dags/SUM.log s3://pd24/savedata/jai/SUM.log
    aws s3api put-object --bucket pd24 --key savedata/jai/DONE
    """,
    retries=1,
    retry_delay=timedelta(minutes=10),
    trigger_rule=TriggerRule.ALL_SUCCESS,
    dag=dag
)

# Set task dependencies
copy_web_log_task >> create_logs_task >> send_logs_task